# Vector RAG Pipeline
1. QA pipeline with LangChain
2. Persistent vector storage based on ChromaDB
3. Evaluation with ragas

In [1]:
import os
from dotenv import load_dotenv

import huggingface_hub
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
if os.path.exists('private-config.env'):
    load_dotenv('private-config.env', override=True)

In [3]:
hf_token = os.getenv('HF_TOKEN')
# Use the token with the Hugging Face library
huggingface_hub.login(token=hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# from langchain_huggingface import HuggingFaceEmbeddings

# model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": False}
# embeddings = HuggingFaceEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

embeddings = OpenAIEmbeddings()

In [6]:
# Test LLM
response = llm.invoke("Hello, how are you?")
response

AIMessage(content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e9b18a6-29ff-407b-b1d3-bf6138383198-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
sample_docs = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine."
]

In [ ]:
documents = [Document(page_content=doc) for doc in sample_docs]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
docstore = text_splitter.split_documents(documents)

# Create ChromaDB and store embeddings
db = Chroma.from_documents(docstore, embeddings, persist_directory="./chroma_db")

# Create a retriever
retriever = db.as_retriever(search_kwargs={"k": 5})

/var/folders/pq/pzj9g_pd2vl6fsmjt6t01msw0000gn/T/ipykernel_97516/133973202.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [9]:
# Querying ChromaDB to Get Context
query = "What is the key takeaway from the document?"
retrieved_docs = retriever.invoke(query)
retrieved_context = "\n\n".join([doc.page_content for doc in retrieved_docs])
print("Retrieved Context:\n", retrieved_context)

Retrieved Context:
 'On the Origin of Species'.

'On the Origin of Species'.

'On the Origin of Species'.

the foundation for classical mechanics.

the foundation for classical mechanics.


In [ ]:
# Define an LLM and Generate Response Using Context
prompt = f"""
Based on the following context, answer the question:\n
Context: \n
{retrieved_context}\n
Question: {query}\n
Answer:
"""
response = llm.invoke(prompt)
print("\nGenerated Response:\n", response.content)


Generated Response:
 The key takeaway from "On the Origin of Species" is the theory of evolution by natural selection, which explains how species adapt and evolve over time through the process of variation and survival of the fittest.


In [11]:
sample_queries = [
    "Who introduced the theory of relativity?",
    "Who was the first computer programmer?",
    "What did Isaac Newton contribute to science?",
    "Who won two Nobel Prizes for research on radioactivity?",
    "What is the theory of evolution by natural selection?"
]

expected_responses = [
    "Albert Einstein proposed the theory of relativity, which transformed our understanding of time, space, and gravity.",
    "Ada Lovelace is regarded as the first computer programmer for her work on Charles Babbage's early mechanical computer, the Analytical Engine.",
    "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "Marie Curie was a physicist and chemist who conducted pioneering research on radioactivity and won two Nobel Prizes.",
    "Charles Darwin introduced the theory of evolution by natural selection in his book 'On the Origin of Species'."
]

In [12]:
from tqdm import tqdm
dataset = []

for query,reference in tqdm(zip(sample_queries,expected_responses)):

    relevant_docs = retriever.invoke(query)
    relevant_context = "\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"Based on the following context, answer the question:\n\n{relevant_context}\n\nQuestion: {query}\nAnswer:"
    response = llm.invoke(prompt).content
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":[d.page_content for d in relevant_docs],
            "response":response,
            "reference":reference
        }
    )

5it [00:05,  1.04s/it]


In [13]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [14]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, AnswerCorrectness, AnswerRelevancy, ContextRecall, ContextPrecision

evaluator_llm = LangchainLLMWrapper(llm)

metrics_list = [
    LLMContextRecall(),
    Faithfulness(),
    FactualCorrectness(),
    AnswerCorrectness(),
    AnswerRelevancy(),
    ContextRecall(),
    ContextPrecision()
]
result = evaluate(dataset=evaluation_dataset,metrics=metrics_list,llm=evaluator_llm)
result

Evaluating:   0%|          | 0/35 [00:00<?, ?it/s]

{'context_recall': 1.0000, 'faithfulness': 0.9000, 'factual_correctness': 0.7800, 'answer_correctness': 0.6608, 'answer_relevancy': 0.9554, 'context_precision': 0.7717}

In [39]:
import numpy as np 

def pca(data: np.ndarray, k: int) -> np.ndarray:
    n, d = data.shape
    data = (data - data.mean(axis=0)) / np.std(data, axis=0)
    cov = data.T @ data / (n-1)
    eigval, eigvec = np.linalg.eigh(cov)
    idx = np.argsort(eigval)[::-1]
    print(eigval, eigvec)
    principal_components = eigvec[:, idx[:k]]
    return np.round(principal_components, 4)

print(pca(np.array([[4,2,1],[5,6,7],[9,12,1],[4,6,7]]),2))

[0.02069312 1.1799001  2.79940678] [[-0.72394434  0.07764021 -0.68547545]
 [ 0.63643914  0.45855781 -0.62021767]
 [-0.26617629  0.88526647  0.3813836 ]]
[[-0.6855  0.0776]
 [-0.6202  0.4586]
 [ 0.3814  0.8853]]


In [38]:

import numpy as np

def pca(data, k):
    # Standardize the data
    data_standardized = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
    
    # Compute the covariance matrix
    covariance_matrix = np.cov(data_standardized, rowvar=False)
    
    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    print(eigenvalues, eigenvectors)
    
    # Sort the eigenvectors by decreasing eigenvalues
    idx = np.argsort(eigenvalues)[::-1]
    eigenvalues_sorted = eigenvalues[idx]
    eigenvectors_sorted = eigenvectors[:,idx]
    
    # Select the top k eigenvectors (principal components)
    principal_components = eigenvectors_sorted[:, :k]
    
    return np.round(principal_components, 4)

data = np.array([[4,2,1],[5,6,7],[9,12,1],[4,6,7]])

print(pca(data, 2))

[2.79940678 0.02069312 1.1799001 ] [[ 0.68547545  0.72394434  0.07764021]
 [ 0.62021767 -0.63643914  0.45855781]
 [-0.3813836   0.26617629  0.88526647]]
[[ 0.6855  0.0776]
 [ 0.6202  0.4586]
 [-0.3814  0.8853]]
